In [1]:
# Do all imports and installs here
from pyspark.sql.functions import udf
from pyspark.sql.session import SparkSession
import datetime
import pandas as pd
import pyspark.sql.functions as F
import psycopg2
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.types import DateType
from pyspark.sql import types as T


In [2]:
spark = SparkSession.builder\
                    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:2.7.0")\
                    .appName("Project: i94")\
                    .getOrCreate()

In [3]:
dir_read = 'D:/Capstone-Project/Project-Workspace/sas_data_part-00000'
dir_write = 'D:/Capstone-Project/Project-Workspace/json_data_part-00000/stg_cic'

In [4]:
sp_sas_data = spark.read.parquet(dir_read)

In [5]:
# drop columns that CIC does not use
sp_sas_data = sp_sas_data.drop("dtadfile","visapost","occup","entdepa","entdepd","entdepu","dtaddto")

In [6]:
sp_sas_data.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [7]:
# Create a function
def convert_5_digit_sasdate_to_yyyymmdd(sasdate):
    epoch = datetime.datetime(1960, 1, 1)
    return (epoch + datetime.timedelta(days=sasdate)).strftime('%Y%m%d')

# Create the UDF
convert_5_digit_sasdate_to_yyyymmdd_udf = udf(lambda x: convert_5_digit_sasdate_to_yyyymmdd(x))

print(convert_5_digit_sasdate_to_yyyymmdd(20574.0))


20160430


In [8]:
# Create the UDF
func_udf =  udf(lambda x: datetime.datetime.strptime(x, '%Y%m%d'), TimestampType())

print(func_udf('20160430'))

Column<b'<lambda>(20160430)'>


In [9]:
sp_df = sp_sas_data
#sp_df = sp_df.drop("_c0")
# drop columns that CIC does not use
sp_df = sp_df.drop("dtadfile","visapost","occup","entdepa","entdepd","entdepu","dtaddto")
sp_df = sp_df.drop("i94yr","i94mon","insnum","admnum")
sp_df = sp_df.withColumnRenamed("count", "i94count")



In [10]:
sp_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- i94count: double (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [11]:
sp_depdate_nn = sp_df.where(F.col("depdate").isNotNull())
sp_depdate_nn = sp_depdate_nn.withColumn('arrdate', convert_5_digit_sasdate_to_yyyymmdd_udf(F.col('arrdate')))
sp_depdate_nn = sp_depdate_nn.withColumn('depdate', convert_5_digit_sasdate_to_yyyymmdd_udf(F.col('depdate')))


In [12]:
sp_depdate_nn.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- i94count: double (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [13]:
#sp_depdate_nn = sp_depdate_nn.withColumn('arrdate', func_udf(F.col('arrdate')))
#sp_depdate_nn = sp_depdate_nn.withColumn('depdate', func_udf(F.col('depdate')))


In [14]:
sp_depdate_nn.createOrReplaceTempView("depdate_nn")
spark.sql("""
select *
from depdate_nn
where 1 = 1
""").show(5)

+-----+------+------+-------+--------+-------+-------+--------+------+-------+--------+-------+-------+------+-------+-----+--------+
|cicid|i94cit|i94res|i94port| arrdate|i94mode|i94addr| depdate|i94bir|i94visa|i94count|matflag|biryear|gender|airline|fltno|visatype|
+-----+------+------+-------+--------+-------+-------+--------+------+-------+--------+-------+-------+------+-------+-----+--------+
| 15.0| 101.0| 101.0|    WAS|20160401|    1.0|     MI|20160825|  55.0|    2.0|     1.0|      M| 1961.0|     M|     OS|   93|      B2|
| 16.0| 101.0| 101.0|    NYC|20160401|    1.0|     MA|20160423|  28.0|    2.0|     1.0|      M| 1988.0|  null|     AA|00199|      B2|
| 17.0| 101.0| 101.0|    NYC|20160401|    1.0|     MA|20160423|   4.0|    2.0|     1.0|      M| 2012.0|  null|     AA|00199|      B2|
| 18.0| 101.0| 101.0|    NYC|20160401|    1.0|     MI|20160411|  57.0|    1.0|     1.0|      M| 1959.0|  null|     AZ|00602|      B1|
| 19.0| 101.0| 101.0|    NYC|20160401|    1.0|     NJ|20160414

In [15]:
sp_depdate_n = sp_df.where(F.col('depdate').isNull())
sp_depdate_n = sp_depdate_n.withColumn('arrdate', convert_5_digit_sasdate_to_yyyymmdd_udf(F.col('arrdate')))
#sp_depdate_n = sp_depdate_n.withColumn('arrdate', func_udf(F.col('arrdate')))
#sp_depdate_n = sp_depdate_n.withColumn('depdate', F.col('depdate').cast(TimestampType()))


In [16]:
sp_depdate_n.createOrReplaceTempView("depdate_n")
spark.sql("""
select *
from depdate_n
where 1 = 1
""").show(5)

+-----+------+------+-------+--------+-------+-------+-------+------+-------+--------+-------+-------+------+-------+-----+--------+
|cicid|i94cit|i94res|i94port| arrdate|i94mode|i94addr|depdate|i94bir|i94visa|i94count|matflag|biryear|gender|airline|fltno|visatype|
+-----+------+------+-------+--------+-------+-------+-------+------+-------+--------+-------+-------+------+-------+-----+--------+
|  6.0| 692.0| 692.0|    XXX|20160429|   null|   null|   null|  37.0|    2.0|     1.0|   null| 1979.0|  null|   null| null|      B2|
|  7.0| 254.0| 276.0|    ATL|20160407|    1.0|     AL|   null|  25.0|    3.0|     1.0|   null| 1991.0|     M|   null|00296|      F1|
| 34.0| 101.0| 101.0|    NYC|20160401|    1.0|     CT|   null|  48.0|    2.0|     1.0|   null| 1968.0|     M|     AZ|00602|      B2|
| 35.0| 101.0| 101.0|    NYC|20160401|    1.0|     CT|   null|  74.0|    2.0|     1.0|   null| 1942.0|     F|     TK|    1|      B2|
| 61.0| 102.0| 102.0|    MIA|20160401|    2.0|     FL|   null|  47.0|

In [17]:
sp_stg_cic = sp_depdate_nn.union(sp_depdate_n)


In [18]:
sp_stg_cic.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- i94count: double (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [19]:
sp_stg_cic.createOrReplaceTempView("stg_cic")

In [20]:
spark.sql("""
select *
from stg_cic
where 1 = 1
""").show(5)

+-----+------+------+-------+--------+-------+-------+--------+------+-------+--------+-------+-------+------+-------+-----+--------+
|cicid|i94cit|i94res|i94port| arrdate|i94mode|i94addr| depdate|i94bir|i94visa|i94count|matflag|biryear|gender|airline|fltno|visatype|
+-----+------+------+-------+--------+-------+-------+--------+------+-------+--------+-------+-------+------+-------+-----+--------+
| 15.0| 101.0| 101.0|    WAS|20160401|    1.0|     MI|20160825|  55.0|    2.0|     1.0|      M| 1961.0|     M|     OS|   93|      B2|
| 16.0| 101.0| 101.0|    NYC|20160401|    1.0|     MA|20160423|  28.0|    2.0|     1.0|      M| 1988.0|  null|     AA|00199|      B2|
| 17.0| 101.0| 101.0|    NYC|20160401|    1.0|     MA|20160423|   4.0|    2.0|     1.0|      M| 2012.0|  null|     AA|00199|      B2|
| 18.0| 101.0| 101.0|    NYC|20160401|    1.0|     MI|20160411|  57.0|    1.0|     1.0|      M| 1959.0|  null|     AZ|00602|      B1|
| 19.0| 101.0| 101.0|    NYC|20160401|    1.0|     NJ|20160414

In [21]:
spark.sql("""
select *
from stg_cic
where 1 = 1
and depdate is not null
""").show(5)

+-----+------+------+-------+--------+-------+-------+--------+------+-------+--------+-------+-------+------+-------+-----+--------+
|cicid|i94cit|i94res|i94port| arrdate|i94mode|i94addr| depdate|i94bir|i94visa|i94count|matflag|biryear|gender|airline|fltno|visatype|
+-----+------+------+-------+--------+-------+-------+--------+------+-------+--------+-------+-------+------+-------+-----+--------+
| 15.0| 101.0| 101.0|    WAS|20160401|    1.0|     MI|20160825|  55.0|    2.0|     1.0|      M| 1961.0|     M|     OS|   93|      B2|
| 16.0| 101.0| 101.0|    NYC|20160401|    1.0|     MA|20160423|  28.0|    2.0|     1.0|      M| 1988.0|  null|     AA|00199|      B2|
| 17.0| 101.0| 101.0|    NYC|20160401|    1.0|     MA|20160423|   4.0|    2.0|     1.0|      M| 2012.0|  null|     AA|00199|      B2|
| 18.0| 101.0| 101.0|    NYC|20160401|    1.0|     MI|20160411|  57.0|    1.0|     1.0|      M| 1959.0|  null|     AZ|00602|      B1|
| 19.0| 101.0| 101.0|    NYC|20160401|    1.0|     NJ|20160414

In [22]:
spark.sql("""
select *
from stg_cic
where 1 = 1
and depdate is null
""").show(5)

+-----+------+------+-------+--------+-------+-------+-------+------+-------+--------+-------+-------+------+-------+-----+--------+
|cicid|i94cit|i94res|i94port| arrdate|i94mode|i94addr|depdate|i94bir|i94visa|i94count|matflag|biryear|gender|airline|fltno|visatype|
+-----+------+------+-------+--------+-------+-------+-------+------+-------+--------+-------+-------+------+-------+-----+--------+
|  6.0| 692.0| 692.0|    XXX|20160429|   null|   null|   null|  37.0|    2.0|     1.0|   null| 1979.0|  null|   null| null|      B2|
|  7.0| 254.0| 276.0|    ATL|20160407|    1.0|     AL|   null|  25.0|    3.0|     1.0|   null| 1991.0|     M|   null|00296|      F1|
| 34.0| 101.0| 101.0|    NYC|20160401|    1.0|     CT|   null|  48.0|    2.0|     1.0|   null| 1968.0|     M|     AZ|00602|      B2|
| 35.0| 101.0| 101.0|    NYC|20160401|    1.0|     CT|   null|  74.0|    2.0|     1.0|   null| 1942.0|     F|     TK|    1|      B2|
| 61.0| 102.0| 102.0|    MIA|20160401|    2.0|     FL|   null|  47.0|

In [23]:
sp_stg_cic.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- i94count: double (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [24]:
sp_stg_cic = sp_stg_cic.withColumn('cicid', F.col('cicid').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('i94cit', F.col('i94cit').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('i94res', F.col('i94res').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('i94mode', F.col('i94mode').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('i94bir', F.col('i94bir').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('i94visa', F.col('i94visa').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('i94count', F.col('i94count').cast(IntegerType()))
sp_stg_cic = sp_stg_cic.withColumn('biryear', F.col('biryear').cast(IntegerType()))


In [25]:
sp_stg_cic.printSchema()

root
 |-- cicid: integer (nullable = true)
 |-- i94cit: integer (nullable = true)
 |-- i94res: integer (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: string (nullable = true)
 |-- i94mode: integer (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: string (nullable = true)
 |-- i94bir: integer (nullable = true)
 |-- i94visa: integer (nullable = true)
 |-- i94count: integer (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- fltno: string (nullable = true)
 |-- visatype: string (nullable = true)



In [26]:
df = sp_stg_cic.select(sp_stg_cic.cicid
                       ,sp_stg_cic.i94cit
                       ,sp_stg_cic.i94res
                       ,sp_stg_cic.i94port
                       ,sp_stg_cic.arrdate
                       ,sp_stg_cic.i94mode
                       ,sp_stg_cic.i94addr
                       ,sp_stg_cic.depdate
                       ,sp_stg_cic.i94bir
                       ,sp_stg_cic.i94visa
                       ,sp_stg_cic.i94count
                       ,sp_stg_cic.matflag
                       ,sp_stg_cic.biryear
                       ,sp_stg_cic.airline
                       ,sp_stg_cic.gender
                       ,sp_stg_cic.fltno
                       ,sp_stg_cic.visatype
                       )

In [27]:
df.createOrReplaceTempView("df")

In [28]:
spark.sql("""
select count(*)
from df
where 1 = 1
""").show()

+--------+
|count(1)|
+--------+
|  219268|
+--------+



In [29]:
df.coalesce(1).write.format('json').mode('overwrite').save(dir_write)